In [25]:
import numpy as np
import os
import cv2
import glob
from enum import Enum

# Drawing mode enum
class DrawOptions(Enum):
    AXES = 1
    CUBE = 2

# Draw the XYZ axes on the image
def drawAxes(img, corners, imgpts):
    def tupleOfInts(arr):
        return tuple(int(x) for x in arr)

    corner = tupleOfInts(corners[0].ravel())
    img = cv2.line(img, corner, tupleOfInts(imgpts[0].ravel()), (255, 0, 0), 5)  # X - Blue
    img = cv2.line(img, corner, tupleOfInts(imgpts[1].ravel()), (0, 255, 0), 5)  # Y - Green
    img = cv2.line(img, corner, tupleOfInts(imgpts[2].ravel()), (0, 0, 255), 5)  # Z - Red
    return img

# Draw a 3D cube on the chessboard
def drawCube(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)

    # Draw bottom face
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), -3)

    # Draw vertical lines
    for i in range(4):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[i + 4]), (255, 255, 255), 3)

    # Draw top face
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)
    return img

# Main pose estimation function
def poseEstimation(option: DrawOptions):
    # Load camera parameters
    calib_data_path = r"C:\Users\youssef\Desktop\CALIBRATIONS\exract_intrinsic&&extrinsic_parameters\calib_data\MultiMatrix.npz"
    data = np.load(calib_data_path)
    camMatrix = data['camMatrix']
    distCoeff = data['distCoef']  # <-- Adjust this based on your actual key

    # Chessboard configuration
    nRows, nCols = 7, 7
    square_size = 25  # mm
    worldPts = np.zeros((nRows * nCols, 3), np.float32)
    worldPts[:, :2] = np.mgrid[0:nRows, 0:nCols].T.reshape(-1, 2) * square_size

    # 3D object shapes
    axis = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]) * square_size
    cube = np.float32([
        [0, 0, 0], [0, 3, 0], [3, 3, 0], [3, 0, 0],
        [0, 0, -3], [0, 3, -3], [3, 3, -3], [3, 0, -3]
    ]) * square_size

    # Load calibration images
    image_dir = r"C:\Users\youssef\Desktop\CALIBRATIONS\Calibartion_Camera_with_checkerboard\images"

    image_paths = glob.glob(os.path.join(image_dir, '*.png'))

    if not image_paths:
        print("❌ No images found in directory.")
        return

    # Create output directory
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)

    # Termination criteria for cornerSubPix
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"❌ Failed to load image: {img_path}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        found, corners = cv2.findChessboardCorners(gray, (nRows, nCols), None)
        if not found:
            print(f"⚠️ Chessboard not found in: {os.path.basename(img_path)}")
            continue

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        ret, rvecs, tvecs = cv2.solvePnP(worldPts, corners2, camMatrix, distCoeff)

        if option == DrawOptions.AXES:
            imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, camMatrix, distCoeff)
            img = drawAxes(img, corners2, imgpts)
        elif option == DrawOptions.CUBE:
            imgpts, _ = cv2.projectPoints(cube, rvecs, tvecs, camMatrix, distCoeff)
            img = drawCube(img, imgpts)

        # Show the result
        window_name = f'Pose Estimation - {os.path.basename(img_path)}'
        cv2.imshow(window_name, img)
        cv2.waitKey(1000)  # Show for 1 second
        cv2.destroyWindow(window_name)

        # Save result image
        save_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(save_path, img)
        print(f"✅ Saved: {save_path}")

    cv2.destroyAllWindows()

# Interactive entry point
if __name__ == "__main__":
    while True:
        user_input = input("Enter 'a' for AXES, 'c' for CUBE, or 'q' to quit: ").lower()
        if user_input == 'a':
            poseEstimation(DrawOptions.AXES)
        elif user_input == 'c':
            poseEstimation(DrawOptions.CUBE)
        elif user_input == 'q':
            print("✅ Exiting.")
            break
        else:
            print("❌ Invalid input. Try again.")


✅ Saved: output\checkerboard_0.png
✅ Saved: output\checkerboard_1.png
✅ Saved: output\checkerboard_10.png
✅ Saved: output\checkerboard_11.png
✅ Saved: output\checkerboard_2.png
✅ Saved: output\checkerboard_3.png
✅ Saved: output\checkerboard_4.png
✅ Saved: output\checkerboard_5.png
✅ Saved: output\checkerboard_6.png
✅ Saved: output\checkerboard_7.png
✅ Saved: output\checkerboard_8.png
✅ Saved: output\checkerboard_9.png
✅ Exiting.
